In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/training/

Mounted at /content/drive
'new 1.py'		       news-commentary-v9.fr-en.en
 news-commentary-v9.cs-en.cs   news-commentary-v9.fr-en.fr
 news-commentary-v9.cs-en.en   news-commentary-v9.ru-en.en
 news-commentary-v9.de-en.de   news-commentary-v9.ru-en.ru
 news-commentary-v9.de-en.en


In [16]:
with open('/content/drive/MyDrive/training/news-commentary-v9.fr-en.en','rb') as f:
  train_en=f.readlines()
with open('/content/drive/MyDrive/training/news-commentary-v9.fr-en.fr','rb') as f:
  train_fr=f.readlines()
print(len(train_en),len(train_fr))
max=0
for j,i in enumerate(train_en+train_fr):
  if len(i.decode('utf-8').split())>max:
    max=len(i.decode('utf-8').split())
print(max)
MAX_LENGTH=max

183251 183251
223


In [17]:
%matplotlib inline
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [18]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [19]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):    
    s=s.lower().strip()
    s=s.decode('utf-8')
    s = unicodeToAscii(s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [20]:
pairs = [[normalizeString(i),normalizeString(j)] for i,j in zip(train_fr,train_en)]
len(pairs)

183251

In [22]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for i,j in kf.split(pairs):
  print(len(i),len(j))
 

146600 36650
146600 36650
146600 36650
146600 36650
146600 36650


In [23]:
#input_lang = Lang('ru')
#output_lang = Lang('eng')
#print("Read %s sentence pairs" % len(pairs))
#pairs = filterPairs(pairs)
#print("Trimmed to %s sentence pairs" % len(pairs))
#num_test = int(len(pairs)/5)
#print(num_test)
#l1=pairs[:num_test]
#l2=pairs[num_test:num_test*2]
#l3=pairs[num_test*2:num_test*3]
#l4=pairs[num_test*3:num_test*4]
#l5=pairs[num_test*4:]
def manage_test_data(test_pairs):
  set_test_eng = set([sent_eng for sent_eng, _ in test_pairs])

  test_pair_dict = {}
  for sent_eng, sent_fre in pairs:
      if sent_eng not in set_test_eng:
          continue 
      elif sent_eng not in test_pair_dict:
          test_pair_dict[sent_eng] = set([sent_fre])
      else:
          test_pair_dict[sent_eng].add(sent_fre)
  return [(sent_eng, list(test_pair_dict[sent_eng])) for sent_eng in test_pair_dict]

#test_pairs = manage_test_data(pairs[132480:])
#test_pairs=test_pairs[:10000]
#train_pairs= pairs[:132480]

#print("Number of test pairs:", len(test_pairs))
#print("Number of train pairs:", len(train_pairs))

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        #previous attention
        #attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        #updated attention
        attn_weights=torch.mm(hidden[0],torch.transpose(encoder_outputs,0,1))
        attn_weights=F.softmax(attn_weights,dim=1)
        #print('dot')
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output=torch.tanh(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ') if word in lang.word2index]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def trainIters(encoder, decoder, n_iters,train_pairs,input_lang,output_lang, print_every=1000, plot_every=100, learning_rate=0.01):
    #train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang,
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print('in train iters',len(train_pairs))
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    #print(training_pairs[0])
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [26]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [27]:
from nltk.translate.bleu_score import corpus_bleu

def evaluateBleu(encoder, decoder, test_pairs):
    references, candidates = [], []    
    for sents_src, sents_tgt in test_pairs:      
      tgt_words    = [sent_tgt.split(' ') for sent_tgt in sents_tgt]      
      src_words, _ = evaluate(encoder, decoder, sents_src)      
      references.append(tgt_words)
      candidates.append(src_words)      
    score1 = corpus_bleu(references, candidates,weights=(1,0,0,0))
    score2 = corpus_bleu(references, candidates,weights=(0,1,0,0))
    score3 = corpus_bleu(references, candidates,weights=(0,0,1,0))
    return score1,score2,score3

Training and Evaluating

**For BLEU 1,2,3 of ru-en language pair**

In [76]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('ru')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]  
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 132480 181 sentence pairs
Counting words...
fr 3844
eng 41694
in train iters 132480
0m 40s (- 2m 43s) (1000 20%) 6.2533
1m 20s (- 2m 0s) (2000 40%) 5.5759
2m 0s (- 1m 20s) (3000 60%) 5.6116
2m 40s (- 0m 40s) (4000 80%) 5.4094
3m 21s (- 0m 0s) (5000 100%) 5.3844


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.2863908817683863 0.05485402291611322 0.0025155024163445294
Read 132481 157 sentence pairs
Counting words...
fr 4762
eng 45120
in train iters 132481
0m 42s (- 2m 49s) (1000 20%) 6.0333
1m 24s (- 2m 7s) (2000 40%) 5.6838
2m 6s (- 1m 24s) (3000 60%) 5.4562
2m 47s (- 0m 41s) (4000 80%) 5.2750
3m 28s (- 0m 0s) (5000 100%) 5.3395
BLEUs : 0.31268436578171094 0.054260187404663325 0.005415162454873647
Read 132481 162 sentence pairs
Counting words...
fr 4762
eng 45120
in train iters 132481
0m 43s (- 2m 54s) (1000 20%) 6.1350
1m 26s (- 2m 9s) (2000 40%) 5.7209
2m 8s (- 1m 25s) (3000 60%) 5.5221
2m 49s (- 0m 42s) (4000 80%) 5.3807
3m 29s (- 0m 0s) (5000 100%) 5.3250
BLEUs : 0.10468851575560771 0.01516900517141835 0.0008070405388944209
Read 132481 176 sentence pairs
Counting words...
fr 4762
eng 45120
in train iters 132481
0m 42s (- 2m 49s) (1000 20%) 6.0829
1m 22s (- 2m 4s) (2000 40%) 5.5884
2m 3s (- 1m 22s) (3000 60%) 5.4119
2m 43s (- 0m 40s) (4000 80%) 5.3255
3m 23s (- 0m 0s) (5000 100

**For BLEU 1,2,3 of du-en langiage pair**

In [15]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('de')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]   
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 161029 4997 sentence pairs
Counting words...
de 118742
eng 44415
in train iters 161029
1m 57s (- 7m 48s) (1000 20%) 6.2713
3m 49s (- 5m 43s) (2000 40%) 5.7174
5m 43s (- 3m 48s) (3000 60%) 5.5913
7m 35s (- 1m 53s) (4000 80%) 5.4654
9m 27s (- 0m 0s) (5000 100%) 5.4647


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.11897042577912327 0.005711477265097199 6.7948607239028e-05
Read 161029 4990 sentence pairs
Counting words...
de 133183
eng 47972
in train iters 161029
2m 4s (- 8m 17s) (1000 20%) 6.1413
4m 8s (- 6m 12s) (2000 40%) 5.7439
6m 11s (- 4m 7s) (3000 60%) 5.6276
8m 14s (- 2m 3s) (4000 80%) 5.4535
10m 19s (- 0m 0s) (5000 100%) 5.4451


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.12276689389905117 0.006092877275113411 0.47677939397031427
Read 161030 4995 sentence pairs
Counting words...
de 133183
eng 47972
in train iters 161030
2m 6s (- 8m 24s) (1000 20%) 6.1407
4m 6s (- 6m 10s) (2000 40%) 5.6893
6m 8s (- 4m 5s) (3000 60%) 5.4162
8m 12s (- 2m 3s) (4000 80%) 5.5531
10m 15s (- 0m 0s) (5000 100%) 5.2900
BLEUs : 0.11242322817657735 0.007585324085566408 0.00018200616275003905
Read 161030 4999 sentence pairs
Counting words...
de 133183
eng 47972
in train iters 161030
1m 59s (- 7m 58s) (1000 20%) 6.0631
4m 2s (- 6m 3s) (2000 40%) 5.6640
6m 4s (- 4m 2s) (3000 60%) 5.6013
8m 5s (- 2m 1s) (4000 80%) 5.4380
10m 8s (- 0m 0s) (5000 100%) 5.3157
BLEUs : 0.10328018204346809 0.005359924819564254 9.946336325276445e-05
Read 161030 4979 sentence pairs
Counting words...
de 133183
eng 47972
in train iters 161030
2m 2s (- 8m 9s) (1000 20%) 6.2685
4m 6s (- 6m 9s) (2000 40%) 5.8153
6m 7s (- 4m 5s) (3000 60%) 5.4325
8m 10s (- 2m 2s) (4000 80%) 5.5408
10m 13s (- 0m 0s) (5000 1

In [28]:
BLEU_1,BLEU_2,BLEU_3=[],[],[]
input_lang = Lang('fr')
output_lang = Lang('eng')
for train_index,test_index in kf.split(pairs):
  train_pairs=[pairs[i] for i in train_index]
  test_pairs=[pairs[i] for i in test_index]   
  print("Read %s %s sentence pairs" % (len(train_pairs),len(test_pairs)))
  print("Counting words...")
  for pair in train_pairs:
      input_lang.addSentence(pair[0])
      output_lang.addSentence(pair[1])
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  hidden_size = 256
  encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
  attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
  trainIters(encoder1, attn_decoder1, 5000,
             train_pairs=train_pairs,input_lang=input_lang,output_lang=output_lang, 
             print_every=1000)
  b1,b2,b3=evaluateBleu(encoder1, attn_decoder1,test_pairs)
  print('BLEUs :',b1,b2,b3)
  BLEU_1.append(b1)
  BLEU_2.append(b2)
  BLEU_3.append(b3)
print('BLEU1: %s BLEU2: %s BLEU3: %s' % (sum(BLEU_1)/5,sum(BLEU_2)/5,sum(BLEU_3)/5))  

Read 146600 4985 sentence pairs
Counting words...
fr 57681
eng 42690
in train iters 146600
1m 42s (- 6m 48s) (1000 20%) 7.7877
3m 20s (- 5m 0s) (2000 40%) 6.8483
4m 55s (- 3m 16s) (3000 60%) 6.4108
6m 34s (- 1m 38s) (4000 80%) 6.5830
8m 8s (- 0m 0s) (5000 100%) 6.9365


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.015434669316228078 0.0003697778128381926 7.483797578243099e-06
Read 146600 4964 sentence pairs
Counting words...
fr 62401
eng 46072
in train iters 146600
1m 38s (- 6m 34s) (1000 20%) 6.2916
3m 16s (- 4m 54s) (2000 40%) 5.6953
4m 53s (- 3m 15s) (3000 60%) 5.5235
6m 31s (- 1m 37s) (4000 80%) 5.4890
8m 9s (- 0m 0s) (5000 100%) 5.5130


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEUs : 0.08343462637514165 0.000557176324653699 0.38604461683666463
Read 146600 4995 sentence pairs
Counting words...
fr 62401
eng 46072
in train iters 146600
1m 38s (- 6m 35s) (1000 20%) 6.1147
3m 19s (- 4m 58s) (2000 40%) 5.7626
4m 58s (- 3m 18s) (3000 60%) 5.4702
6m 36s (- 1m 39s) (4000 80%) 5.3678
8m 16s (- 0m 0s) (5000 100%) 5.3886
BLEUs : 0.0363931669360752 0.0003002347063923325 4.116158992689613e-06
Read 146600 4997 sentence pairs
Counting words...
fr 62401
eng 46072
in train iters 146600
1m 39s (- 6m 36s) (1000 20%) 6.2886
3m 17s (- 4m 56s) (2000 40%) 5.6852
4m 52s (- 3m 15s) (3000 60%) 5.5411
6m 30s (- 1m 37s) (4000 80%) 5.5520
8m 9s (- 0m 0s) (5000 100%) 5.4654
BLEUs : 0.12475306040922203 0.003223810059637672 5.78841044460781e-06
Read 146600 4982 sentence pairs
Counting words...
fr 62401
eng 46072
in train iters 146600
1m 37s (- 6m 28s) (1000 20%) 6.1097
3m 12s (- 4m 48s) (2000 40%) 5.5871
4m 53s (- 3m 15s) (3000 60%) 5.6466
6m 32s (- 1m 38s) (4000 80%) 5.4239
8m 10s (- 0m 0